In [1]:
from sparktorch import serialize_torch_obj, SparkTorch
import torch
import torch.nn as nn

In [2]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, lit

In [3]:
from pyspark.sql.types import IntegerType

In [4]:
from pyspark.ml.feature import VectorAssembler

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master(
    'local[4]'
).config(
    "spark.driver.memory",'2g'
).config(
    "spark.executor.memory",'1g'
).appName('abc').getOrCreate()

22/06/13 23:16:13 WARN Utils: Your hostname, JUANs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.10.30 instead (on interface en0)
22/06/13 23:16:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/13 23:16:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
train = '../data/04_model_input/train'
cv = '../data/04_model_input/cv'
test = '../data/04_model_input/test'

In [7]:
train_df = spark.read.parquet(train)
cv_df = spark.read.parquet(cv)
test_df = spark.read.parquet(test)

In [8]:
def inner_join(df):
    df = df.drop('address','number')
    df_1 = df.select(*(col(x).alias(x + '_x') for x in df.columns)).withColumn('join', lit(1))
    df_2 = df.select(*(col(x).alias(x + '_y') for x in df.columns)).withColumn('join', lit(1))
    return df_1.join(
        df_2, on='join'
    ).withColumn(
        'same',(col('name_x')==col('name_y')).astype(IntegerType())
    ).drop('join','name_x','name_y')

In [15]:
train_join = inner_join(train_df.sample(False,0.001))
cv_join = inner_join(cv_df)

In [17]:
train_join.write.parquet(train+'_join')

22/06/13 23:17:59 WARN DAGScheduler: Broadcasting large task binary with size 1117.5 KiB


In [10]:
in_cols = [c for c in train_join.columns if c !='same']

In [11]:
vector_assembler = VectorAssembler(inputCols=in_cols,outputCol='features',)

In [12]:
train_join = vector_assembler.transform(train_join).select('features','same')
cv_join = vector_assembler.transform(cv_join).select('features','same')

In [13]:
train_join

DataFrame[features: vector, same: int]

In [14]:
network = nn.Sequential(
    nn.Linear(len(in_cols), 20),
    nn.ReLU(),
    nn.Linear(20, 5),
    nn.Softmax(dim=1)
)

In [15]:
torch_obj = serialize_torch_obj(
    model=network,
    criterion=nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam,
    lr=0.0001
)


In [16]:
spark_model = SparkTorch(
    inputCol='features',
    labelCol='same',
    predictionCol='predictions',
    torchObj=torch_obj,
    iters=50,
    verbose=1,
    miniBatch=100,
)

In [17]:
spark_model.fit(train_join)

22/06/13 23:04:28 WARN DAGScheduler: Creating new stage failed due to exception - job: 3
org.apache.spark.scheduler.BarrierJobUnsupportedRDDChainException: [SPARK-24820][SPARK-24821]: Barrier execution mode does not allow the following pattern of RDD chain within a barrier stage:
1. Ancestor RDDs that have different number of partitions from the resulting RDD (e.g. union()/coalesce()/first()/take()/PartitionPruningRDD). A workaround for first()/take() can be barrierRdd.collect().head (scala) or barrierRdd.collect()[0] (python).
2. An RDD that depends on multiple barrier RDDs (e.g. barrierRdd1.zip(barrierRdd2)).
	at org.apache.spark.scheduler.DAGScheduler.checkBarrierStageWithRDDChainPattern(DAGScheduler.scala:447)
	at org.apache.spark.scheduler.DAGScheduler.createResultStage(DAGScheduler.scala:590)
	at org.apache.spark.scheduler.DAGScheduler.handleJobSubmitted(DAGScheduler.scala:1196)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:2592)
	at o

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.scheduler.BarrierJobUnsupportedRDDChainException: [SPARK-24820][SPARK-24821]: Barrier execution mode does not allow the following pattern of RDD chain within a barrier stage:
1. Ancestor RDDs that have different number of partitions from the resulting RDD (e.g. union()/coalesce()/first()/take()/PartitionPruningRDD). A workaround for first()/take() can be barrierRdd.collect().head (scala) or barrierRdd.collect()[0] (python).
2. An RDD that depends on multiple barrier RDDs (e.g. barrierRdd1.zip(barrierRdd2)).
	at org.apache.spark.scheduler.DAGScheduler.checkBarrierStageWithRDDChainPattern(DAGScheduler.scala:447)
	at org.apache.spark.scheduler.DAGScheduler.createResultStage(DAGScheduler.scala:590)
	at org.apache.spark.scheduler.DAGScheduler.handleJobSubmitted(DAGScheduler.scala:1196)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.

In [ ]:

from tensorflow import keras
from tensorflow.keras import layers

In [37]:
input_faces = keras.Input(shape = (len(in_cols)))

In [38]:
dense_1 = layers.Dense(20, activation='relu')(input_faces)
predictions = layers.Dense(1, activation='sigmoid')(dense_1)

Metal device set to: Apple M1


2022-06-13 22:03:35.927074: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-13 22:03:35.927386: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [39]:
model = keras.Model(inputs=input_faces, outputs=predictions)

In [40]:
model.compile(
  'adam',
  loss='binary_crossentropy',
  metrics=['accuracy'],
)

In [16]:
train_join

NameError: name 'train_join' is not defined

In [46]:
from keras.models import model_from_yaml, slice_X

ImportError: cannot import name 'slice_X' from 'keras.models' (/Users/juanazuero/miniforge3/envs/face_pairing/lib/python3.10/site-packages/keras/models/__init__.py)

In [14]:
from elephas.spark_model import SparkModel

ImportError: cannot import name 'slice_X' from 'keras.models' (/Users/juanazuero/miniforge3/envs/face_pairing/lib/python3.10/site-packages/keras/models/__init__.py)

In [ ]:


spark_model = SparkModel(model, frequency='epoch', mode='asynchronous')
spark_model.fit(rdd, epochs=20, batch_size=32, verbose=0, validation_split=0.1)

In [5]:
from elephas.ml_model import ElephasEstimator

ImportError: cannot import name 'keyword_only' from 'pyspark.ml.util' (/Users/juanazuero/miniforge3/envs/face_pairing/lib/python3.10/site-packages/pyspark/ml/util.py)

In [ ]:
from pyspark.ml.util import 

In [11]:
import elephas

In [12]:
elephas.__version__

AttributeError: module 'elephas' has no attribute '__version__'

In [10]:

from elephas.utils.rdd_utils import to_simple_rdd
from elephas.spark_model import SparkModel

ImportError: cannot import name 'slice_X' from 'keras.models' (/Users/juanazuero/miniforge3/envs/face_pairing/lib/python3.10/site-packages/keras/models/__init__.py)

In [9]:
!pip install --upgrade elephas

  Using cached elephas-3.1.0.tar.gz (26 kB)
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.30-py2.py3-none-any.whl (985 kB)
  Using cached elephas-3.0.0.tar.gz (26 kB)
  Preparing metadata (setup.py) ... done
  Using cached elephas-2.1.0.tar.gz (27 kB)
  Preparing metadata (setup.py) ... done
  Using cached elephas-2.0.0.tar.gz (26 kB)
  Preparing metadata (setup.py) ... done
  Using cached elephas-1.4.3.tar.gz (30 kB)
  Preparing metadata (setup.py) ... done
  Using cached elephas-1.4.2.tar.gz (30 kB)
  Preparing metadata (setup.py) ... done
  Using cached elephas-1.4.1.tar.gz (30 kB)
  Preparing metadata (setup.py) ... done
  Using cached elephas-1.3.1.tar.gz (30 kB)
  Preparing metadata (setup.py) ... done
  Using cached elephas-1.2.1.tar.gz (30 kB)
  Preparing metadata (setup.py) ... done
  Using cached elephas-1.2.0.tar.gz (29 kB)
  Preparing metadata (setup.py) ... done
  Using cached elephas-1.1.0.tar.gz (29 kB)
  Preparing metadata (setup.py) ... done
  Usi